In [2]:
import pandas as pd
import numpy as np
import datetime
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [99]:
train = pd.read_csv('PJT002_train.csv')
test = pd.read_csv('PJT002_test.csv')
validation = pd.read_csv('PJT002_validation.csv')

C:\Users\lee16\Anaconda3\envs\tensorflow3.5\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [100]:
train = pd.concat([train.loc[:,"id":"hmdt"],train.loc[:,"jmk":"fr_yn"]],axis=1)
validation = pd.concat([validation.loc[:,"id":"hmdt"],validation.loc[:,"jmk":"fr_yn"]],axis=1)
test = pd.concat([test.loc[:,"id":"hmdt"],test.loc[:,"jmk":"bldng_cnt_in_50m"]],axis=1)

In [101]:
y_train = train['fr_yn']
x_train = train.drop(['fr_yn','dt_of_fr','bldng_archtctr', 'dt_of_athrztn', 'bldng_us_clssfctn',
       'jmk', 'rgnl_ar_nm', 'rgnl_ar_nm2', 'lnd_us_sttn_nm', 'rd_sd_nm',
       'emd_nm', 'mlt_us_yn','bldng_us'], axis = 1)

y_val = validation['fr_yn']
x_val = validation.drop(['fr_yn','dt_of_fr','bldng_archtctr', 'dt_of_athrztn', 'bldng_us_clssfctn',
       'jmk', 'rgnl_ar_nm', 'rgnl_ar_nm2', 'lnd_us_sttn_nm', 'rd_sd_nm',
       'emd_nm', 'mlt_us_yn','bldng_us'], axis = 1)

test = test.drop(['dt_of_fr','bldng_archtctr', 'dt_of_athrztn', 'bldng_us_clssfctn',
       'jmk', 'rgnl_ar_nm', 'rgnl_ar_nm2', 'lnd_us_sttn_nm', 'rd_sd_nm',
       'emd_nm', 'mlt_us_yn','bldng_us'], axis = 1)

In [102]:
pre = pd.concat([x_train,x_val,test],axis=0)

In [112]:
pre = pre.reset_index(drop=True)

In [114]:
pre.shape

(69054, 25)

In [117]:
print(x_train.shape)
print(x_val.shape)
print(test.shape)

(59199, 25)
(6898, 25)
(2957, 25)


In [120]:
data2 = pd.concat([pre,one_data],axis=1)

In [125]:
x_train = data2.loc[0:59198]
x_val = data2.loc[59198+1:59198+6898]
test = data2.loc[59198+6898+1:59198+6898+2957]

In [55]:
one_data

['bldng_us',
 'bldng_archtctr',
 'dt_of_athrztn',
 'bldng_us_clssfctn',
 'jmk',
 'rgnl_ar_nm',
 'rgnl_ar_nm2',
 'lnd_us_sttn_nm',
 'rd_sd_nm',
 'emd_nm',
 'mlt_us_yn']

In [38]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [42]:
pre['bldng_us'].dtype

dtype('O')

In [44]:
list(pre['bldng_us'])

[nan,
 '업무시설',
 '단독주택',
 '위험물저장및처리시설',
 nan,
 '공동주택',
 '위험물저장및처리시설',
 '단독주택',
 '단독주택',
 '단독주택',
 '제1종근린생활시설',
 '단독주택',
 nan,
 '단독주택',
 nan,
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 '공장',
 nan,
 '제2종근린생활시설',
 '제2종근린생활시설',
 '노유자시설',
 '단독주택',
 '공동주택',
 '단독주택',
 '제1종근린생활시설',
 '제2종근린생활시설',
 nan,
 nan,
 '단독주택',
 nan,
 nan,
 nan,
 nan,
 '공동주택',
 '단독주택',
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 '제1종근린생활시설',
 '단독주택',
 '운동시설',
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 '단독주택',
 nan,
 nan,
 '제1종근린생활시설',
 '단독주택',
 '제2종근린생활시설',
 '단독주택',
 nan,
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 nan,
 '제1종근린생활시설',
 '단독주택',
 '공동주택',
 '공장',
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 '위험물저장및처리시설',
 '단독주택',
 nan,
 '공동주택',
 '단독주택',
 '단독주택',
 '단독주택',
 '제2종근린생활시설',
 '공동주택',
 nan,
 '동.식물 관련시설',
 '단독주택',
 '단독주택',
 '동.식물 관련시설',
 '단독주택',
 '단독주택',
 nan,
 '단독주택',
 nan,
 nan,
 nan,
 '공장',
 '단독주택',
 '단독주택',
 '단독주택',
 '제

In [69]:
encoder = LabelEncoder()
encoder.fit(list(pre['bldng_us']))
labels = encoder.transform(list(pre['bldng_us']))
labels = labels.reshape(-1,1)

one_hot = OneHotEncoder()
one_hot.fit(labels)
one_hot_labels = one_hot.transform(labels).toarray()

In [72]:
one_data = pd.DataFrame(one_hot_labels)

In [88]:
col_a = []
for i in range(35):
    a = str('bldng_us')+str(i)
    col_a.append(a)

In [89]:
one_data.columns = col_a

In [90]:
one_data

,bldng_us0,bldng_us1,bldng_us2,bldng_us3,bldng_us4,bldng_us5,bldng_us6,bldng_us7,bldng_us8,bldng_us9,...,bldng_us25,bldng_us26,bldng_us27,bldng_us28,bldng_us29,bldng_us30,bldng_us31,bldng_us32,bldng_us33,bldng_us34
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
ohe = OneHotEncoder()
encoder = LabelEncoder()
onehot_encoder_list = []
for column_name in list(factor_columns):
    encoder.fit(list(pre[column_name]))
    labels = encoder.transform(list(pre[column_name]))
    labels = labels.reshape(-1,1)
    ohe = OneHotEncoder()
    ohe.fit(labels)
    onehot_encoder_list.append(ohe) #각 컬럼 별로 label encoder 저장
    del ohe

In [62]:
onehot_encoder_list

[OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
        handle_unknown='error', n_values='auto', sparse=True),
 OneHotEncoder(categorical_features='all'

In [4]:
y_train = train['fr_yn']
x_train = train.drop(['fr_yn'], axis = 1)

y_val = validation['fr_yn']
x_val = validation.drop(['fr_yn'], axis = 1)

In [139]:
def xxx(x):
    if x == "Y":
        return 1
    else:
        return 0
    
y_train = y_train.apply(lambda x: xxx(x))
y_val = y_val.apply(lambda x: xxx(x))

In [140]:
train_ds = lgb.Dataset(x_train, label = y_train)
val_ds = lgb.Dataset(x_val, label = y_val)

In [141]:
params = {'learning_rate': 0.01,
          'max_depth': 16,
          'boosting': 'gbdt',
          'objective': 'binary',
          'metric': 'binary_error',
          'is_training_metric': True,
          'num_leaves': 144,
          'feature_fraction': 0.9,
          'bagging_fraction': 0.7,
          'bagging_freq': 5,
          'seed':2019}

In [143]:
model = lgb.train(params, train_ds, 1000, val_ds, verbose_eval=10, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[10]	valid_0's binary_error: 0.184401
[20]	valid_0's binary_error: 0.184401
[30]	valid_0's binary_error: 0.184401
[40]	valid_0's binary_error: 0.184401
[50]	valid_0's binary_error: 0.184401
[60]	valid_0's binary_error: 0.184401
[70]	valid_0's binary_error: 0.184401
[80]	valid_0's binary_error: 0.184401
[90]	valid_0's binary_error: 0.184401
[100]	valid_0's binary_error: 0.184401
Early stopping, best iteration is:
[1]	valid_0's binary_error: 0.184401


In [146]:
list(model.predict(test))

[1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
 1.0000000036274914e-15,
